In [ ]:
# Installing libraries
!pip install segmentation-models==1.0.1
!pip install tensorflow
!pip install h5py
!pip install SimpleITK

In [ ]:
# Importing libraries
%matplotlib inline
import glob
import cv2
import random
import os
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
%env SM_FRAMEWORK=tf.keras
import segmentation_models as sm
import pandas as pd
import imageio
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from scipy.stats import sem, t
from numpy import mean
from tensorflow.keras.layers import Input
from keras.models import Model
import scipy.stats as stats
from keras.layers import Average
from keras.utils import to_categorical

In [ ]:
# mounting google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Loading images and masks for the BML model (Model 3)

mask_dir = '/content/drive/MyDrive/BML_Masks'
img_dir = '/content/drive/MyDrive/BML_Images'

# Initialize lists to store the medial and lateral images and masks
img_list = []
mask_list = []

# Initialize a list to store image names
image_names = []

# Sort the subject subfolders in each directory
img_subjects = sorted(os.listdir(img_dir))
mask_subjects = sorted(os.listdir(mask_dir))

# Loop over the subject subfolders in the image directory
for subject_id in img_subjects:
    # Set the path to the subject's subfolder
    subject_img_dir = os.path.join(img_dir, subject_id)

    # Get a list of image files in the subject's subfolder
    img_files = [f for f in os.listdir(subject_img_dir) if f.endswith('.tif')]

    # Sort the image files by their slice number
    img_files.sort(key=lambda x: int(x.split('.')[0]))

    # Loop over the sorted image files
    for img_file in img_files:
        # Set the path to the image file
        img_path = os.path.join(subject_img_dir, img_file)

        # Load the image using imageio
        img = imageio.v2.imread(img_path)
        slice_number = int(img_file.split('.')[0])

        #image padding
        pad_height = 448 - img.shape[0]
        pad_width = 448 - img.shape[1]
        # Apply padding to height and width
        padding = ((0, pad_height), (0, pad_width))
        img = np.pad(img, padding, mode='constant', constant_values=0)
        image_name = f"{subject_id}_{slice_number}"
        # Add image name to list
        image_names.append(image_name)


        # Add the image to the medial image list
        img_list.append(img)

# Convert the image lists to numpy arrays
img_array = np.array(img_list)
image_names = np.array(image_names)

# Loop over the subject subfolders in the mask directory
for subject_id in mask_subjects:
    # Set the path to the subject's subfolder
    subject_mask_dir = os.path.join(mask_dir, subject_id)

    # Get a list of mask files in the subject's subfolder
    mask_files = [f for f in os.listdir(subject_mask_dir) if f.endswith('.tif')]

    # Sort the mask files by their slice number
    mask_files.sort(key=lambda x: int(x.split('.')[0]))

    # Loop over the sorted mask files
    for mask_file in mask_files:
        # Set the path to the mask file
        mask_path = os.path.join(subject_mask_dir, mask_file)

        # Load the mask using imageio
        mask = imageio.v2.imread(mask_path)

        # Check if the slice number is less than 16 or greater than 18
        slice_number = int(mask_file.split('.')[0])
        #image padding
        pad_height = 448 - mask.shape[0]
        pad_width = 448 - mask.shape[1]
        # Apply padding to height and width
        padding = ((0, pad_height), (0, pad_width))
        mask = np.pad(mask, padding, mode='constant', constant_values=0)

        # Add the mask to the medial mask list
        mask_list.append(mask)

# Convert the mask list to numpy array
mask_array = np.array(mask_list)

In [ ]:
# Encoding labels for combined BML masks
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
n, h, w = mask_array.shape
train_masks_reshaped = mask_array.reshape(-1,1)
train_masks_reshaped_encoded = labelencoder.fit_transform(train_masks_reshaped)
train_masks_encoded_original_shape = train_masks_reshaped_encoded.reshape(n, h, w)
print(train_masks_encoded_original_shape.shape)
print(train_masks_reshaped_encoded.shape)
np.unique(train_masks_encoded_original_shape)

In [ ]:
# Adding a dimension to masks to match images (Read in RGB)
train_masks_input = np.expand_dims(train_masks_encoded_original_shape, axis=3)

# Reformating classes
n_classes = 2
train_masks_cat = to_categorical(train_masks_input, num_classes=n_classes)
masks_cat = train_masks_cat.reshape((train_masks_input.shape[0], train_masks_input.shape[1], train_masks_input.shape[2], n_classes))

In [ ]:
# Define a learning rate schedule
def scheduler(epoch, lr):
    if epoch < 50:
        return lr
    else:
        return lr * tf.math.exp(-0.1)

# Create a callback for the learning rate scheduler
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [ ]:
#---------------kfold cross validation the model-------------------


# Define the number of folds
n_folds = 5

# Define the dice coefficient function
def dice_coefficient(y_true, y_pred):
    intersection = np.sum(y_true * y_pred, axis=(1, 2))
    union = np.sum(y_true, axis=(1, 2)) + np.sum(y_pred, axis=(1, 2))
    dice = 2. * intersection / union
    return dice

# Initialize the cross-validation object
kf = KFold(n_splits=n_folds)

# Initialize a list to store the history of each fold
models = []

# Loop over each fold
for fold, (train_index, val_index) in enumerate(kf.split(img_array_rgb)):
    # Split the data
    X_train, X_val = img_array_rgb[train_index], img_array_rgb[val_index]
    y_train, y_val = masks_cat[train_index], masks_cat[val_index]

    #Define BML_fold model
    activation='softmax'
    LR = 0.0001
    optim = tf.keras.optimizers.Adam(LR)
    total_loss = sm.losses.DiceLoss()
    metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5), 'accuracy']
    BACKBONE = 'resnet50'
    preprocess_input = sm.get_preprocessing(BACKBONE)
    X_train = preprocess_input(X_train)
    X_val = preprocess_input(X_val)
    BMLmodel_fold = sm.Unet(BACKBONE, encoder_weights='imagenet', classes=n_classes, activation=activation, decoder_use_batchnorm=True)
    BMLmodel_fold.compile(optimizer=optim, loss=total_loss, metrics=metrics)

    # Fit the model
    history = BMLmodel_fold.fit(X_train,
                           y_train,
                           batch_size=8,
                           epochs=100,
                           verbose=1,
                           validation_data=(X_val, y_val),
                           callbacks=[callback])


    # Add the trained model to the list
    models.append(BMLmodel_fold)

# Create an ensemble model from the 5 models
ensemble_inputs = Input(shape=img_array_rgb.shape[1:])
ensemble_outputs = Average()([model(ensemble_inputs) for model in models])
ensemble_model = Model(inputs=ensemble_inputs, outputs=ensemble_outputs)

# Make predictions on the entire dataset with the ensemble model
y_pred_ensemble = ensemble_model.predict(img_array_rgb)

# Calculate the Dice score and AUC for the ensemble model
dice_scores_ensemble = dice_coefficient(masks_cat[..., 1], y_pred_ensemble[..., 1])
mean_dice_ensemble = np.mean(dice_scores_ensemble)
confidence_interval_dice_ensemble = stats.t.interval(0.95, len(dice_scores_ensemble)-1, loc=mean_dice_ensemble, scale=stats.sem(dice_scores_ensemble))

auc_scores_ensemble = [roc_auc_score(masks_cat[i, ..., 1].flatten(), y_pred_ensemble[i, ..., 1].flatten()) for i in range(masks_cat.shape[0])]
mean_auc_ensemble = np.mean(auc_scores_ensemble)
confidence_interval_auc_ensemble = stats.t.interval(0.95, len(auc_scores_ensemble)-1, loc=mean_auc_ensemble, scale=stats.sem(auc_scores_ensemble))

# Calculate and print the Dice score and AUC for the ensemble model
print("Ensemble model Dice score: ", mean_dice_ensemble)
print("Ensemble model 95% confidence interval for Dice score: ", confidence_interval_dice_ensemble)
print("Ensemble model AUC: ", mean_auc_ensemble)
print("Ensemble model 95% confidence interval for AUC: ", confidence_interval_auc_ensemble)

In [ ]:
# Saving the model
model.save('/content/drive/MyDrive/BML_segmentation/BMLmodel.h5')

In [ ]:
#-----------------------------leave one center out cross validation model-----------------------

# Define the dice coefficient function
def dice_coefficient(y_true, y_pred):
    intersection = np.sum(y_true * y_pred, axis=(1, 2))
    union = np.sum(y_true, axis=(1, 2)) + np.sum(y_pred, axis=(1, 2))
    dice = 2. * intersection / union
    return dice

# Initialize lists to store the Dice scores, AUCs, and models for each center
dice_scores = []
aucs = []
models = []

center_names = ['A', 'B', 'C', 'D', 'E']  # Define your center names

for center in center_names:
    # Create a boolean mask for the current center
    train_mask = [name.split('_')[1] != center for name in names]
    test_mask = [name.split('_')[1] == center for name in names]

    # Apply the mask to get the train and test sets for the current center
    X_train_center = img_array_rgb[train_mask]
    y_train_center = masks_cat[train_mask]
    X_test_center = img_array_rgb[test_mask]
    y_test_center = masks_cat[test_mask]

    # Clone the base model for the current center
    activation='softmax'
    LR = 0.0001
    optim = tf.keras.optimizers.Adam(LR)
    total_loss = sm.losses.DiceLoss()
    metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5), 'accuracy']
    BACKBONE = 'resnet50'
    preprocess_input = sm.get_preprocessing(BACKBONE)
    X_train_center = preprocess_input(X_train_center)
    X_test_center = preprocess_input(X_test_center)
    BMLmodel_center = sm.Unet(BACKBONE, encoder_weights='imagenet', classes=n_classes, activation=activation, decoder_use_batchnorm=True)
    BMLmodel_center.compile(optimizer=optim, loss=total_loss, metrics=metrics)

    # Fit the model for the current center
    history = BMLmodel_center.fit(X_train_center,
                                  y_train_center,
                                  batch_size=8,
                                  epochs=100,
                                  verbose=1,
                                  validation_data=(X_test_center, y_test_center),callbacks=[callback])

    # Add the trained model to the list
    models.append(BMLmodel_center)

# Create an ensemble model from the 5 models
ensemble_inputs = Input(shape=img_array_rgb.shape[1:])
ensemble_outputs = Average()([model(ensemble_inputs) for model in models])
ensemble_model = Model(inputs=ensemble_inputs, outputs=ensemble_outputs)

# Make predictions on the entire dataset with the ensemble model
y_pred_ensemble = ensemble_model.predict(img_array_rgb)

# Calculate the Dice score and AUC for the ensemble model
dice_scores_ensemble = dice_coefficient(masks_cat[..., 1], y_pred_ensemble[..., 1])
mean_dice_ensemble = np.mean(dice_scores_ensemble)
confidence_interval_dice_ensemble = stats.t.interval(0.95, len(dice_scores_ensemble)-1, loc=mean_dice_ensemble, scale=stats.sem(dice_scores_ensemble))

auc_scores_ensemble = [roc_auc_score(masks_cat[i, ..., 1].flatten(), y_pred_ensemble[i, ..., 1].flatten()) for i in range(masks_cat.shape[0])]
mean_auc_ensemble = np.mean(auc_scores_ensemble)
confidence_interval_auc_ensemble = stats.t.interval(0.95, len(auc_scores_ensemble)-1, loc=mean_auc_ensemble, scale=stats.sem(auc_scores_ensemble))

# Calculate and print the Dice score and AUC for the ensemble model
print("Ensemble model Dice score: ", mean_dice_ensemble)
print("Ensemble model 95% confidence interval for Dice score: ", confidence_interval_dice_ensemble)
print("Ensemble model AUC: ", mean_auc_ensemble)
print("Ensemble model 95% confidence interval for AUC: ", confidence_interval_auc_ensemble)